# Baselining Pressure Data: Pump 5

## 1. Import required libraries

In [1]:
%load_ext skip_kernel_extension

In [2]:
import pandas as pd, numpy as np
import os, requests, datetime
import plotly.express as px
from plotly.offline import init_notebook_mode
import plotly.io as pio
init_notebook_mode(connected = True)
pio.templates.default = "simple_white"

from fn_data_ops_v3 import *

## Inputs 

**Provide required inputs below-**

`skip_baseline_save == False` <font color='red'>*>> To save baseline data to DB.</font> <br>
`skip_baseline_save == True` <font color='red'>*>> To run the notebook for baseline week selection.</font>

In [3]:
skip_baseline_save = False  # To skip baseline save, set it as True

In [4]:
data_folder =  "..\Data\\Pressure-Data\\Pump5\\"                        # Pressure data folder name
asset_name = 'g-pump-5'                                                 # Asset Name
asset_id = fn_get_asset_id(asset_name)                                  # Asset Id
datestring_column_name = "DateString"                                   # Date column name in the data

sensor_type = "pressure"                                                
HOUSE_PRESSURE = "HousePressure"
INLET_PRESSURE = "InletPressure"
OUTLET_PRESSURE = "OutletPressure"
measurement_type_housing = 'housing'
measurement_type_inlet = 'inlet'
measurement_type_outlet = 'outlet'

In [5]:
# Azure API details 
dict_api = {'hourly_data_api': 'https://hpp-cm-npd-aps-001.azurewebsites.net/AssetHourlyData/',
            'baseline_data_api' : 'https://hpp-cm-npd-aps-001.azurewebsites.net/AssetBaseline/'}

In [6]:
hourly_data_filename = "{}-azure-hourly-data.csv".format(asset_id)      # filename for hourly data
baseline_data_filename = "{}-azure-baseline-data.csv".format(asset_id)  # filename for baselining data
print('Hourly data filename: ',hourly_data_filename)                    
print('Baseline data filename: ', baseline_data_filename)    
print('\nCurrent Working Directory: ', os.getcwd())

Hourly data filename:  aid-g-pump-5-azure-hourly-data.csv
Baseline data filename:  aid-g-pump-5-azure-baseline-data.csv

Current Working Directory:  C:\Users\kumar_mano\OneDrive - HCL Technologies Ltd\Projects\Danfoss\Dan-WD\save_to_api


**Pressure file names:**

In [7]:
for file_ in [OUTLET_PRESSURE, INLET_PRESSURE, HOUSE_PRESSURE]:
    print(get_file_path(data_folder, file_))

..\Data\Pressure-Data\Pump5\OutletPressure.csv
..\Data\Pressure-Data\Pump5\InletPressure.csv
..\Data\Pressure-Data\Pump5\HousePressure.csv


# Baselining Activity

### Processing Hourly Data for Baselining Activity
* Following dataset should be obtained from an API call

### Get Hourly processed data 
* 3 Data Frames for Outlet, Housing and Inlet pressure each.

In [8]:
df_hourly_outletpressure_azure = get_hourly_data_for_azure(get_file_path(data_folder, OUTLET_PRESSURE), asset_id, asset_name, OUTLET_PRESSURE, measurement_type_outlet, datestring_column_name, sensor_type)
df_hourly_inletpressure_azure = get_hourly_data_for_azure(get_file_path(data_folder, INLET_PRESSURE), asset_id, asset_name,  INLET_PRESSURE, measurement_type_inlet, datestring_column_name, sensor_type)
df_hourly_housepressure_azure = get_hourly_data_for_azure(get_file_path(data_folder, HOUSE_PRESSURE),asset_id, asset_name,  HOUSE_PRESSURE, measurement_type_housing, datestring_column_name, sensor_type)

In [9]:
# df_pump_azure_data = df_hourly_housepressure_azure.append(df_hourly_inletpressure_azure).append(df_hourly_outletpressure_azure)
# df_pump_azure_data.reset_index(drop=True, inplace=True)
# df_pump_azure_data.to_csv(hourly_data_filename, index = False)
# df_pump_azure_data.head()
# #df_pump4_azure_data.dtypes

### Filter the Outliers in the Data

##### Remove negative values from hourly pressure data for baselining

In [10]:
print(df_hourly_outletpressure_azure.shape, df_hourly_inletpressure_azure.shape, df_hourly_housepressure_azure.shape)
df_hourly_outletpressure_azure = fn_remove_negative_values(df_= df_hourly_outletpressure_azure)
df_hourly_inletpressure_azure = fn_remove_negative_values(df_= df_hourly_inletpressure_azure)
df_hourly_housepressure_azure = fn_remove_negative_values(df_= df_hourly_housepressure_azure)
print(df_hourly_outletpressure_azure.shape, df_hourly_inletpressure_azure.shape, df_hourly_housepressure_azure.shape)

(2796, 9) (2796, 9) (2796, 9)
(2796, 9) (2796, 9) (2796, 9)


***Filter values less than 5 and greater than 95 percentile from the AvgValue data.***

In [11]:
print(df_hourly_outletpressure_azure.shape, df_hourly_inletpressure_azure.shape, df_hourly_housepressure_azure.shape)

df_hourly_outletpressure_azure = fn_drop_outliers(df_hourly = df_hourly_outletpressure_azure)
df_hourly_inletpressure_azure = fn_drop_outliers(df_hourly = df_hourly_inletpressure_azure)
df_hourly_housepressure_azure = fn_drop_outliers(df_hourly = df_hourly_housepressure_azure)

print(df_hourly_outletpressure_azure.shape, df_hourly_inletpressure_azure.shape, df_hourly_housepressure_azure.shape)

(2796, 9) (2796, 9) (2796, 9)
(2735, 9) (2703, 9) (2705, 9)


In [12]:
df_pump_azure_data = df_hourly_housepressure_azure.append(df_hourly_inletpressure_azure).append(df_hourly_outletpressure_azure)
df_pump_azure_data.reset_index(drop=True, inplace=True)
df_data = add_date_components(df_pump_azure_data, "datetime")
# df_data.dtypes

In [13]:
df_data.head(2)

,asset_id,asset_name,sensor_type,measurement_type,feature_name,feature_value,feature_unit,sensor_info,datetime,d,dayofweek,year,month,hour,minute,second,week
0,aid-g-pump-5,g-pump-5,pressure,housing,MinValue,2.517552,bar,HousePressure,2019-11-19 18:00:00,19,1,2019,11,18,0,0,47
1,aid-g-pump-5,g-pump-5,pressure,housing,MinValue,3.307387,bar,HousePressure,2019-11-20 08:00:00,20,2,2019,11,8,0,0,47


In [14]:
get_number_of_weeks(df_data, variable_name = INLET_PRESSURE, asset_id = asset_id)
get_number_of_weeks(df_data, variable_name = OUTLET_PRESSURE, asset_id = asset_id)
get_number_of_weeks(df_data, variable_name = HOUSE_PRESSURE, asset_id = asset_id)

Number of weeks for InletPressure found:8
Number of weeks for OutletPressure found:8
Number of weeks for HousePressure found:8


## Take a close look at the probable weeks for baselining

**Scatterplot**

In [15]:
show_baselining_plot_azure(df_data, duration = "all", asset_name= asset_name, variable_name= INLET_PRESSURE,
                           plot_type = "scatter")

show_baselining_plot_azure(df_data, duration = "all", asset_name= asset_name, variable_name= OUTLET_PRESSURE,
                          plot_type = "scatter")

show_baselining_plot_azure(df_data, duration = "all", asset_name= asset_name, variable_name= HOUSE_PRESSURE,
                          plot_type = "scatter")

**Boxplot**

In [16]:
show_baselining_plot_azure(df_data, duration = "all", asset_name= asset_name, variable_name= INLET_PRESSURE, plot_type = "box")

show_baselining_plot_azure(df_data, duration = "all", asset_name= asset_name, variable_name= OUTLET_PRESSURE, plot_type = "box")

show_baselining_plot_azure(df_data, duration = "all", asset_name= asset_name, variable_name= HOUSE_PRESSURE, plot_type = "box")

*** 
*** 

*** 

*** 
*** 

# Baselining Week Inputs



In [17]:
week_numbers_inletPressure = [1,2,51,52] 
week_numbers_outletPressure = [48,49,50,52]
week_numbers_housePressure = [1,2,51,52]

***

## Inlet Pressure

In [18]:
%%skip $skip_baseline_save
week_numbers = week_numbers_inletPressure # Week numbers as List 

mu_inletpressure, sigma_inletpressure = get_baselining_variables_azure(df_data, variable_name = INLET_PRESSURE,
                                                                       asset_id = asset_id,
                                                                       lst_filter_values = week_numbers, 
                                                                       filter_column_name = "week",
                                                                       select_column_name= 'feature_value',
                                                                       filter_metric = "AvgValue")
print(mu_inletpressure, sigma_inletpressure)

show_baselining_plot_azure(df_data, duration = week_numbers, asset_name= asset_name, variable_name= INLET_PRESSURE,
                           plot_type = "box")

2.9299781186099083 0.04029619593847434


## Outlet Pressure

In [19]:
%%skip $skip_baseline_save
week_numbers =  week_numbers_outletPressure # Week numbers as List 

mu_outletpressure, sigma_outletpressure = get_baselining_variables_azure(df_data, variable_name = OUTLET_PRESSURE,
                                                                         asset_id = asset_id,
                                                                         lst_filter_values = week_numbers, 
                                                                         filter_column_name = "week",
                                                                         select_column_name= 'feature_value',
                                                                         filter_metric = "AvgValue")
print(mu_outletpressure, sigma_outletpressure)

show_baselining_plot_azure(df_data, duration = week_numbers, asset_name= asset_name, variable_name= OUTLET_PRESSURE,
                           plot_type = "box")

57.22597946124257 0.34913708239349417


## House Pressure

In [20]:
%%skip $skip_baseline_save
week_numbers = week_numbers_housePressure # Week numbers as List 

mu_housepressure, sigma_housepressure = get_baselining_variables_azure(df_data, variable_name = HOUSE_PRESSURE,
                                                                       asset_id = asset_id,
                                                                       lst_filter_values = week_numbers, 
                                                                       filter_column_name = "week",
                                                                       select_column_name= 'feature_value',
                                                                       filter_metric = "AvgValue")
print(mu_housepressure, sigma_housepressure)

show_baselining_plot_azure(df_data, duration = week_numbers, asset_name= asset_name, variable_name= HOUSE_PRESSURE, 
                           plot_type = "box")

3.3617461589336566 0.053946814471695095


***

In [21]:
df_data.head()

,asset_id,asset_name,sensor_type,measurement_type,feature_name,feature_value,feature_unit,sensor_info,datetime,d,dayofweek,year,month,hour,minute,second,week
0,aid-g-pump-5,g-pump-5,pressure,housing,MinValue,2.517552,bar,HousePressure,2019-11-19 18:00:00,19,1,2019,11,18,0,0,47
1,aid-g-pump-5,g-pump-5,pressure,housing,MinValue,3.307387,bar,HousePressure,2019-11-20 08:00:00,20,2,2019,11,8,0,0,47
2,aid-g-pump-5,g-pump-5,pressure,housing,MinValue,2.525183,bar,HousePressure,2019-11-20 14:00:00,20,2,2019,11,14,0,0,47
3,aid-g-pump-5,g-pump-5,pressure,housing,MinValue,2.612943,bar,HousePressure,2019-11-20 15:00:00,20,2,2019,11,15,0,0,47
4,aid-g-pump-5,g-pump-5,pressure,housing,MinValue,2.727411,bar,HousePressure,2019-11-20 16:00:00,20,2,2019,11,16,0,0,47


In [22]:
df_data.feature_name.unique()

array(['MinValue', 'AvgValue', 'MaxValue'], dtype=object)

In [23]:
%%skip $skip_baseline_save

row_housepressure_mean = get_row_dict_baseline(asset_id = asset_id,
                                               asset_name = asset_name,
                                               sensor_type = sensor_type,
                                               measurement_type = measurement_type_housing,
                                               feature_name = 'AvgValue',
                                               metric_name = 'mean',
                                               metric_value = mu_housepressure,
                                               feature_unit = "bar",
                                               sensor_info = HOUSE_PRESSURE,
                                               notes = None)


row_outletpressure_mean = get_row_dict_baseline(asset_id = asset_id,
                                                asset_name = asset_name,
                                                sensor_type = sensor_type,
                                                measurement_type = measurement_type_outlet,
                                                feature_name = 'AvgValue',
                                                metric_name = 'mean',
                                                metric_value = mu_outletpressure,
                                                feature_unit = "bar",
                                                sensor_info = OUTLET_PRESSURE,
                                                notes = None)

row_inletpressure_mean = get_row_dict_baseline(asset_id = asset_id,
                                                asset_name = asset_name,
                                                sensor_type = sensor_type,
                                                measurement_type = measurement_type_outlet,
                                                feature_name = 'AvgValue',
                                                metric_name = 'mean',
                                                metric_value = mu_inletpressure,
                                                feature_unit = "bar",
                                                sensor_info = INLET_PRESSURE,
                                                notes = None)

In [24]:
%%skip $skip_baseline_save

row_housepressure_sigma = get_row_dict_baseline(asset_id = asset_id,
                                               asset_name = asset_name,
                                               sensor_type = sensor_type,
                                               measurement_type = measurement_type_housing,
                                               feature_name = 'AvgValue',
                                               metric_name = 'sd',
                                               metric_value = sigma_housepressure,
                                               feature_unit = "bar",
                                               sensor_info = HOUSE_PRESSURE,
                                               notes = None)


row_outletpressure_sigma = get_row_dict_baseline(asset_id = asset_id,
                                                asset_name = asset_name,
                                                sensor_type = sensor_type,
                                                measurement_type = measurement_type_outlet,
                                                feature_name = 'AvgValue',
                                                metric_name = 'sd',
                                                metric_value = sigma_outletpressure,
                                                feature_unit = "bar",
                                                sensor_info = OUTLET_PRESSURE,
                                                notes = None)

row_inletpressure_sigma = get_row_dict_baseline(asset_id = asset_id,
                                                asset_name = asset_name,
                                                sensor_type = sensor_type,
                                                measurement_type = measurement_type_outlet,
                                                feature_name = 'AvgValue',
                                                metric_name = 'sd',
                                                metric_value = sigma_inletpressure,
                                                feature_unit = "bar",
                                                sensor_info = INLET_PRESSURE,
                                                notes = None)

In [25]:
%%skip $skip_baseline_save
print(row_outletpressure_sigma)

{'asset_id': 'aid-g-pump-5', 'asset_name': 'g-pump-5', 'sensor_type': 'pressure', 'measurement_type': 'outlet', 'feature_name': 'AvgValue', 'metric_name': 'sd', 'metric_value': 0.34913708239349417, 'feature_unit': 'bar', 'sensor_info': 'OutletPressure', 'baseline_datetime': '2020-03-18 16:32:45', 'baseline_validity_month': 3, 'notes': None}


In [26]:
%%skip $skip_baseline_save
df_baseline = pd.DataFrame(columns=['asset_id','asset_name','sensor_type', 'measurement_type', 'feature_name', 'metric_name',
                                    'metric_value', 'feature_unit', 'sensor_info', 'baseline_datetime', 
                                    'baseline_validity_month', 'notes'])
df_baseline = df_baseline.append(row_housepressure_mean, ignore_index=True)
df_baseline = df_baseline.append(row_housepressure_sigma, ignore_index=True)
df_baseline = df_baseline.append(row_outletpressure_mean, ignore_index=True)
df_baseline = df_baseline.append(row_outletpressure_sigma, ignore_index=True)
df_baseline = df_baseline.append(row_inletpressure_mean, ignore_index=True)
df_baseline = df_baseline.append(row_inletpressure_sigma, ignore_index=True)
#df_baseline['uuid'] = [uuid.uuid4() for _ in range(len(df_baseline.index))]
#df_baseline.drop(['asset_name'], axis=1, inplace=True)
df_baseline.to_csv(baseline_data_filename, index = False)

In [27]:
%%skip $skip_baseline_save
df_baseline.dropna(subset=['metric_value'], inplace=True)
print(df_baseline.head(10))

       asset_id asset_name sensor_type measurement_type feature_name  \
0  aid-g-pump-5   g-pump-5    pressure          housing     AvgValue   
1  aid-g-pump-5   g-pump-5    pressure          housing     AvgValue   
2  aid-g-pump-5   g-pump-5    pressure           outlet     AvgValue   
3  aid-g-pump-5   g-pump-5    pressure           outlet     AvgValue   
4  aid-g-pump-5   g-pump-5    pressure           outlet     AvgValue   
5  aid-g-pump-5   g-pump-5    pressure           outlet     AvgValue   

  metric_name  metric_value feature_unit     sensor_info    baseline_datetime  \
0        mean      3.361746          bar   HousePressure  2020-03-18 16:32:45   
1          sd      0.053947          bar   HousePressure  2020-03-18 16:32:45   
2        mean     57.225979          bar  OutletPressure  2020-03-18 16:32:45   
3          sd      0.349137          bar  OutletPressure  2020-03-18 16:32:45   
4        mean      2.929978          bar   InletPressure  2020-03-18 16:32:45   
5        

*** 

## Convert DataFrame to Json and Push to Azure API

In [ ]:
# json_object = convert_baseline_data_to_json(df_baseline)
# json_object

In [ ]:
%%skip $skip_baseline_save

json_object = convert_baseline_data_to_json(df_baseline)
status_code, reason = save_to_api(json_object, dict_api['baseline_data_api'])
print(status_code, reason)

***